In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
import os




In [ ]:
# Paths to your dataset (modify if needed)
train_dir = '/content/drive/MyDrive/train'
test_dir = '/content/drive/MyDrive/test'

# Image data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Loading the images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # VGG16 expects 224x224 input images
    batch_size=32,
    class_mode='binary'  # Assuming binary classification (covid vs normal)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)



Found 2754 images belonging to 2 classes.
Found 1288 images belonging to 2 classes.


In [ ]:
# Loading the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Adding custom classification layers on top of VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# Final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Calculate steps_per_epoch and validation_steps based on dataset size
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = test_generator.samples // test_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=test_generator,
    validation_steps=validation_steps
)

# Save the model
model.save('/content/vgg16_covid19_classifier.h5')

# Optionally: evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Epoch 1/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - accuracy: 0.8544 - loss: 0.8230 - val_accuracy: 0.9937 - val_loss: 0.0525
Epoch 2/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 0.0854 - val_accuracy: 1.0000 - val_loss: 0.0243
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.9442 - loss: 0.1387 - val_accuracy: 0.9867 - val_loss: 0.0406
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9062 - loss: 0.1213 - val_accuracy: 1.0000 - val_loss: 4.3625e-04
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.9532 - loss: 0.1203 - val_accuracy: 0.9930 - val_loss: 0.0240
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9688 - loss: 0.0776 - val_accuracy: 1.0000 - val_loss: 0.0036
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.9551 - loss: 0.1038 - val_accuracy: 0.9953 - val_loss: 0.0209
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - accuracy: 1.0000 - loss: 0.0911 - val_accuracy: 1.0000 -

41/41 ━━━━━━━━━━━━━━━━━━━━ 20s 490ms/step - accuracy: 0.9881 - loss: 0.0276
Test Accuracy: 99.07%
